In [ ]:
# open a .txt containing the extracted comments from MS Word
trackchange_doc = open("all_comments_reconciled_2020-08-12.txt",'r').read().strip().split("\n")

position_of_comments = []
comments_abstract_match = {}

# we loop over the file
for e in range(len(trackchange_doc)):
    element = trackchange_doc[e].strip()
    # if the line indicates a commeent
    if "Commented" == element:
        # we save the line number and the page it refers to
        number = trackchange_doc[e-1].strip()
        if ":" == number[-1] and number[:-1].isnumeric():
            position_of_comments.append(e)
            comments_abstract_match[e] = number[:-1]
            
# this is the number of comments we found
print (len(position_of_comments))

In [2]:
# we save the annotations in a spreadsheet
out = open("annotations.tsv","w")
out.write("comment_id\tabstract_id\tannotator\tscore\tcomment\n")
# this dictionary matches the name of the annotators as MS Word users with their text mention
authors = {"dey,":"Dey,  Gautam", "jessica":"Jessica  Polka","mate":"Mate  Palfy","jonathon":"jonathon coates"}


# we loop over the comments
for c in range(len(position_of_comments)):
    # we retrieve the position of the comment and the related abstract id (its page)
    comment_pos = position_of_comments[c]
    abstract_id = comments_abstract_match[comment_pos]
    
    # here we know where to stop, at the beginning of the next change (or at the end of the doc)
    if c+1 in position_of_comments:
        nextchange = position_of_comments[c+1]
    else:
        nextchange = len(trackchange_doc)+1
    

    
    # this is the "comment id" as in MS Word
    id_ = trackchange_doc[comment_pos+1].strip()
    
    # we try to find the author as, depending on the format is in one of the next lines
    possible_author = trackchange_doc[comment_pos+2].strip().lower()
    check = False
    if possible_author in authors:
        check = True
        # 95% of the time the author is right after and the comments are after the metadata
        author = authors[possible_author]
        annotation = " ".join(trackchange_doc[comment_pos+7:nextchange]).strip().split("Page ")[0].split("page")[0]
    else:
        # we check if it's in the 3rd line
        possible_author = trackchange_doc[comment_pos+3].strip().lower()
        if possible_author in authors:
            check = True
            # in these very rare cases, the annotation is in the line above
            author = authors[possible_author]
            annotation = trackchange_doc[comment_pos+2].strip().lower().split("Page ")[0].split("page")[0]
        else:
            # otherwise we go iteratively for 10 lines and we stop as soon as we find one of the authors
            # these are 5% of the cases
            for i in range(4,14):
                if check is False:
                    possible_author = trackchange_doc[comment_pos+i].strip().lower()
                    if possible_author in authors:
                        check = True
                        author = authors[possible_author]
                        annotation = " ".join(trackchange_doc[comment_pos+2:comment_pos+i]).strip().split("Page ")[0].split("page")[0]
                else:
                    break
    # if we really did not find a match we will add it manually
    if check == False:
        author = "TOBEADDED"
        annotation = " ".join(trackchange_doc[comment_pos+2:comment_pos+i]).strip().split("Page ")[0].split("page")[0]
    if "(" in annotation:
        comment = annotation.split("(")[1].split(")")[0]
    else:
        comment = ""
    score = annotation.split("(")[0]  
    out.write(id_+"\t"+abstract_id+"\t"+author+"\t"+score+"\t"+comment+"\n")

out.close()